In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.metrics import mean_squared_error

df = pd.read_csv("Neighbourhood_model_data.csv")
print(df)

      NEIGHBOURHOOD_NAME  TAX_ASSESSMENT_YEAR  PREVIOUS_MEDIAN_PRICE  \
0          Arbutus-Ridge                 2007                0.80475   
1          Arbutus-Ridge                 2008                1.01700   
2          Arbutus-Ridge                 2009                1.22905   
3          Arbutus-Ridge                 2010                1.23490   
4          Arbutus-Ridge                 2011                1.22900   
5          Arbutus-Ridge                 2012                1.51500   
6          Arbutus-Ridge                 2013                1.90910   
7          Arbutus-Ridge                 2014                1.82600   
8          Arbutus-Ridge                 2015                1.82455   
9          Arbutus-Ridge                 2016                1.98200   
10         Arbutus-Ridge                 2017                2.36600   
11         Arbutus-Ridge                 2018                3.24700   
12         Arbutus-Ridge                 2019                3.1

In [2]:
df.describe()

,TAX_ASSESSMENT_YEAR,PREVIOUS_MEDIAN_PRICE,INTEREST_RATE,CURRENT_MEDIAN_PRICE
count,273.000000,273.000000,273.000000,273.000000
mean,2013.000000,1.141014,3.185256,1.240245
std,3.748529,0.847415,1.006144,0.893613
min,2007.000000,0.300000,2.395833,0.362000
25%,2010.000000,0.580900,2.700000,0.628500
50%,2013.000000,0.817000,2.775000,0.907700
75%,2016.000000,1.433950,3.000000,1.517000
max,2019.000000,5.321850,6.104167,5.321850


In [3]:
### constants
area_list = df.NEIGHBOURHOOD_NAME.unique()
area_list

array(['Arbutus-Ridge', 'Downtown', 'Dunbar-Southlands', 'Fairview',
       'Grandview-Woodland', 'Hastings-Sunrise',
       'Kensington-Cedar Cottage', 'Kerrisdale', 'Killarney', 'Kitsilano',
       'Marpole', 'Mount Pleasant', 'Oakridge', 'Renfrew-Collingwood',
       'Riley Park', 'Shaughnessy', 'South Cambie', 'Sunset',
       'Victoria-Fraserview', 'West End', 'West Point Grey'], dtype=object)

In [14]:
import statsmodels.api as sm
import statsmodels
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse_list = []

for area in area_list:
    filtered_df = df.query('NEIGHBOURHOOD_NAME == @area')
    truncated_df = filtered_df.drop(['NEIGHBOURHOOD_NAME', 'PREVIOUS_MEDIAN_PRICE'], axis = 1)
    
    sales_series = df['CURRENT_MEDIAN_PRICE']
    lag_series = df['INTEREST_RATE']

    train_sales = sales_series[:int(0.8*(len(sales_series)))]
    valid_sales = sales_series[int(0.8*(len(sales_series))):]

    lag_train = lag_series[:int(0.8*(len(lag_series)))]
    lag_valid = lag_series[int(0.8*(len(lag_series))):]

    model4=sm.tsa.ARIMA(endog=train_sales.values,exog=lag_train.values,order=[1,0,2])
    results4=model4.fit()
    
    prediction = results4.forecast(exog=lag_valid, steps=len(lag_valid))
    pred = pd.DataFrame(prediction[0])

    rmse = sqrt(mean_squared_error(pred, valid_sales))
    rmse_list.append(rmse)

In [15]:
import numpy as np

average_rmse = np.mean(rmse_list)
average_rmse

0.7356704532715904